In [1]:
import json

with open("data2/news.json.2", "r") as f:
    news = json.load(f)


In [2]:
import numpy

n = len(news)
idx = numpy.random.randint(0, n)
html_text = news[idx]["html"]
html_text

'<!DOCTYPE html><html lang="ja"><head><title>3500万の住宅ローンを組んだ「年収700万の夫婦」、“老後破綻の危機”に陥ったワケ（現代ビジネス） - Yahoo!ニュース</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"/><meta name="viewport" content="width=1010"/><meta http-equiv="Content-Style-Type" content="text/css"/><meta http-equiv="Content-Script-Type" content="text/javascript"/><meta name="description" content="\u30003500万円の住宅ローンを組んでマイホームを手に入れ、繰り上げ返済にいそしんできた伊藤孝さん(40歳・仮名、以下同じ)、知子さん(39歳)夫婦。\n\n"/><meta name="robots" content="noarchive, max-image-preview:large"/><meta name="format-detection" content="telephone=no"/><meta name="google-site-verification" content="dVyTbsDzUrUAjCE1aCH9hKwZ8sOsCsvi2uhuBvasrp8"/><meta property="og:site_name" content="Yahoo!ニュース"/><meta property="og:image" content="https://newsatcl-pctr.c.yimg.jp/t/amd-img/20221230-00104178-gendaibiz-000-1-view.jpg?exp=10800"/><meta property="og:image:height" content="340"/><meta property="og:image:width" content="298"/><meta property="og:description" conte

In [ ]:
# with open("data/sample.html", "w") as f:
#     f.write(html_text)


In [3]:
from bs4 import BeautifulSoup

def _filter_text(body) -> str:
    soup: BeautifulSoup = BeautifulSoup(body, "lxml")
    for tg in ["script", "noscript", "meta"]:
        try:
            soup.find(tg).replace_with(" ")
        except Exception:
            # NOTE: Not Found `tg` tag
            pass
    return soup.get_text("\n\n")



In [4]:
text = _filter_text(html_text)
text

'3500万の住宅ローンを組んだ「年収700万の夫婦」、“老後破綻の危機”に陥ったワケ（現代ビジネス） - Yahoo!ニュース\n\n \n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nYahoo!ニュース\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nYahoo! JAPAN\n\n\n\n\n\n\n\nヘルプ\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nキーワード：\n\n\n\n\n\n\n\n検索\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nIDでもっと便利に\n\n新規取得\n\n\n\n\nログイン\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nマイページ\n\n購入履歴\n\nトップ\n\n速報\n\nライブ\n\n個人\n\nオリジナル\n\nみんなの意見\n\nランキング\n\n有料\n\n主要\n\n国内\n\n国際\n\n経済\n\nエンタメ\n\nスポーツ\n\nIT\n\n科学\n\nライフ\n\n地域\n\nトピックス一覧\n\n \n\n3500万の住宅ローンを組んだ「年収700万の夫婦」、“老後破綻の危機”に陥ったワケ\n\n2022/12/30(金)\n\n \n\n6:03\n\n配信\n\n95\n\nコメント\n\n95\n\n件\n\nPhoto by iStock\n\n\u30003500万円の住宅ローンを組んでマイホームを手に入れ、繰り上げ返済にいそしんできた伊藤孝さん(40歳・仮名、以下同じ)、知子さん(39歳)夫婦。\n\n\n\n【マンガ】「一緒にお風呂入ろ」母の再婚相手から性的虐待を受けた女性の罪悪感\n\n\n\n\u3000しかし、【前編】「3500万の住宅ローン組んだ「年収700万夫婦」、繰り上げ返済をして「地獄を見た」ワケ」で詳しく見た通り、教育資金について相談するために、著者であるFPの中村賢司さんのもとを訪れてライ

In [5]:
import re

def clean_text(text: str):
    contents = []
    for txt in re.split(r"(。|\n)", text):
        txt = txt.strip().replace("\u200b", "").replace("\u3000", " ")
        txt = re.sub(r"\n+", "\n", txt)
        txt = re.sub(r"([\W])\1+", " ", txt)
        if not txt:
            continue
        # contents.append(txt)
        # contents.extend(txt.split("\n"))
        contents.append(txt.split("\n")[-1])
    return contents


In [6]:
texts = clean_text(text)
len(texts)
texts[:20]


['3500万の住宅ローンを組んだ「年収700万の夫婦」、“老後破綻の危機”に陥ったワケ（現代ビジネス） - Yahoo!ニュース',
 'Yahoo!ニュース',
 'Yahoo! JAPAN',
 'ヘルプ',
 'キーワード：',
 '検索',
 'IDでもっと便利に',
 '新規取得',
 'ログイン',
 'マイページ',
 '購入履歴',
 'トップ',
 '速報',
 'ライブ',
 '個人',
 'オリジナル',
 'みんなの意見',
 'ランキング',
 '有料',
 '主要']

In [7]:
cleaned_texts = []
for itm in news:
    html = itm["html"]
    text = _filter_text(html_text)
    texts = clean_text(text)
    cleaned_texts.append(texts)

In [8]:
len(cleaned_texts)

4192

In [9]:
cleaned_texts[-1]

['3500万の住宅ローンを組んだ「年収700万の夫婦」、“老後破綻の危機”に陥ったワケ（現代ビジネス） - Yahoo!ニュース',
 'Yahoo!ニュース',
 'Yahoo! JAPAN',
 'ヘルプ',
 'キーワード：',
 '検索',
 'IDでもっと便利に',
 '新規取得',
 'ログイン',
 'マイページ',
 '購入履歴',
 'トップ',
 '速報',
 'ライブ',
 '個人',
 'オリジナル',
 'みんなの意見',
 'ランキング',
 '有料',
 '主要',
 '国内',
 '国際',
 '経済',
 'エンタメ',
 'スポーツ',
 'IT',
 '科学',
 'ライフ',
 '地域',
 'トピックス一覧',
 '3500万の住宅ローンを組んだ「年収700万の夫婦」、“老後破綻の危機”に陥ったワケ',
 '2022/12/30(金)',
 '6:03',
 '配信',
 '95',
 'コメント',
 '95',
 '件',
 'Photo by iStock',
 '3500万円の住宅ローンを組んでマイホームを手に入れ、繰り上げ返済にいそしんできた伊藤孝さん(40歳・仮名、以下同じ)、知子さん(39歳)夫婦',
 '。',
 '【マンガ】「一緒にお風呂入ろ」母の再婚相手から性的虐待を受けた女性の罪悪感',
 'しかし、【前編】「3500万の住宅ローン組んだ「年収700万夫婦」、繰り上げ返済をして「地獄を見た」ワケ」で詳しく見た通り、教育資金について相談するために、著者であるFPの中村賢司さんのもとを訪れてライフプランニングをしてもらうと、あと5年ほどで家計が破綻してしまうことが明らかになりました',
 '。',
 'いったい二人は何を間違ってしまったのでしょうか',
 '。',
 'なぜ家計が破綻してしまうのか',
 '5年後に貯蓄残高がマイナスになる原因のひとつには、知子さんの収入ダウンがありました',
 '。',
 '知子さんは次男が小学校に上がったことをきっかけに、子育てに専念したいと考えはじめ、今年からフルタイムで働くのをやめて夫の扶養の範囲内でパートとして働くようになっていたのです',
 '。',
 '年収は今までの400万円から100万円に減り、これまでのようなペースでは貯蓄ができなくなりました',
 '。

In [19]:
from app.domain.models.tokenizer import TokenizerWord

tokenizer = TokenizerWord(use_stoppoes=False, filterpos=[], use_orgform=False)

In [16]:
tidy_sentences = []
for idx, texts in enumerate(cleaned_texts):
    for text in texts:
        tokenized = tokenizer([[text]])
        if not tokenized:
            continue
        snt = tokenized[0]
        if len(snt) < 3:
            continue
        # print(idx, snt)       # for debugging
        tidy_sentences.append("".join(snt))


In [17]:
len(tidy_sentences)

347936

In [20]:
len(set(tidy_sentences))

72

In [ ]:
tidy_sentences[:100]

In [ ]:
n = len(tidy_sentences)
idx = numpy.random.randint(0, n)
tidy_sentences[idx]

In [ ]:
from app.component.models.pipeline import Pipeline
from app.component.models.vectorizer import VectorizerWord2vec

pipe_vectorizer = Pipeline(
    steps=[
        (
            TokenizerWord(
                use_stoppoes=False, filterpos=["名詞", "動詞"], use_orgform=True
            ),
            None,
        ),
        (VectorizerWord2vec().set_params(dict(
            vector_size=128,
            sg=1,
            max_vocab_size=1000 * 1000,
            min_count=1,
            window=7,
            epochs=50,
        )), None),
    ]
)


In [ ]:
vectorizer:VectorizerWord2vec = pipe_vectorizer.get_model(-1)
vectorizer.params

In [ ]:
tidy_sentences[0]

In [ ]:
len(set(tidy_sentences))

In [ ]:
from app.component.simple_logger import log_info

X = [[s] for s in tidy_sentences]

log_info("Start", "Fit Wiki data")
pipe_vectorizer.fit(X)
log_info("End", "Fit Wiki data")


In [ ]:
vectorizer:VectorizerWord2vec = pipe_vectorizer.get_model(-1)
len(vectorizer.model.wv.index_to_key)


In [ ]:
import joblib

pipe_file = "data/pipe_newsvec.gz"
log_info("Start", "Dump Pipeline for Wiki vectorizer")
joblib.dump(pipe_vectorizer, pipe_file, compress=("gzip", 3))
log_info("End", "Dump Pipeline for Wiki vectorizer")